In [1]:
from keras.layers import Input, Add, Dense, Conv2D, merge, Conv2DTranspose,Concatenate, MaxPooling2D, UpSampling2D, Flatten, Reshape, LSTM
from keras.models import Model

In [3]:
act = 'linear'
input_img = Input(shape=(384,192,1))
input_enc_1 = Input(shape=(1,))
input_enc_2 = Input(shape=(1,))

x1 = Conv2D(16, (3,3),activation=act, padding='same')(input_img)
x1 = MaxPooling2D((2,2),padding='same')(x1)
x1 = Conv2D(8, (3,3),activation=act, padding='same')(x1)
x1 = MaxPooling2D((2,2),padding='same')(x1)
x1 = Conv2D(8, (3,3),activation=act, padding='same')(x1)
x1 = MaxPooling2D((2,2),padding='same')(x1)
x1 = Conv2D(8, (3,3),activation=act, padding='same')(x1)
x1 = MaxPooling2D((2,2),padding='same')(x1)
x1 = Conv2D(4, (3,3),activation=act, padding='same')(x1)
x1 = MaxPooling2D((2,2),padding='same')(x1)
x1 = Conv2D(4, (3,3),activation=act, padding='same')(x1)
x1 = MaxPooling2D((2,2),padding='same')(x1)
x1 = Reshape([3*6*4])(x1)
x1 = Dense(64,activation=act)(x1)
x1 = Dense(32,activation=act)(x1)
x1 = Dense(16,activation=act)(x1)

x1 = Dense(1,activation=act)(x1)
x1 = Concatenate()([x1, input_enc_1,input_enc_2])

x1 = Dense(16,activation=act)(x1)
x1 = Dense(32,activation=act)(x1)
x1 = Dense(64,activation=act)(x1)
x1 = Dense(72,activation=act)(x1)
x1 = Reshape([3,6,4])(x1)
x1 = UpSampling2D((2,2))(x1)
x1 = Conv2D(4, (3,3),activation=act, padding='same')(x1)
x1 = UpSampling2D((2,2))(x1)
x1 = Conv2D(8, (3,3),activation=act, padding='same')(x1)
x1 = UpSampling2D((2,2))(x1)
x1 = Conv2D(8, (3,3),activation=act, padding='same')(x1)
x1 = UpSampling2D((2,2))(x1)
x1 = Conv2D(8, (3,3),activation=act, padding='same')(x1)
x1 = UpSampling2D((2,2))(x1)
x1 = Conv2D(8, (3,3),activation=act, padding='same')(x1)
x1 = UpSampling2D((2,2))(x1)
x1 = Conv2D(16, (3,3),activation=act, padding='same')(x1)

x_final = Conv2D(1, (3,3),padding='same')(x1)
autoencoder = Model([input_img,input_enc_1,input_enc_2], x_final)
autoencoder.compile(optimizer='adam', loss='mse')

In [4]:
autoencoder.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384, 192, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 384, 192, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 192, 96, 16)  0           ['conv2d[0][0]']                 
                                                                                              